In [114]:
from datetime import datetime
import pandas as pd
import numpy as np
from dateutil.parser import parse
import datetime
from dateutil.parser import parse
import math
from numpy import mean

from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_validate, StratifiedKFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline as SKLpipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from dtreeviz.trees import dtreeviz 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as IMBLpipeline

from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import ipynbname
nb_fname = ipynbname.name()

pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 30)

In [115]:
# read df pickle
df_alg = pd.read_pickle("objects/df_alg-HAB_preprocessing_5_1")
data = pd.read_pickle("objects/df_cons4-HAB_preprocessing_5_1")

# data.set_index('date', inplace=True)
# slice by station and time
# df_cons = df_cons[df_cons["sampling station"] == "Debeli_rtic"].loc["2008-01-01" : "2021-12-31"]

data.isnull().sum()

date                       0
sampling station           0
DSP                        1
DSP_like                  60
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    661
salinity                 191
T                        244
SECCHI                   696
DIN                      542
PO4-P                    534
temp_dif                 346
Soca                       0
month                      0
lipophylic_toxins        320
dtype: int64

# Second data preprocessing

## Interpolate closest values

In [116]:
# Interpolate missing values based on closest values at the same location
data_ipl = data.copy()

max_range = pd.Timedelta(30, unit="day")
min_range = pd.Timedelta(-30, unit="day")

# List of features with missing values
features = [
 'Chl-a',
 'salinity',
 'T',
 'SECCHI',
 'DIN',
 'PO4-P',
 ]

# Iterate through all features
for fet in features:
    # Iterate over data and select an index and date of the data instance
    for idx, date in data["date"].iteritems():
        # check if the instance has NaN for this feature
        if pd.isna(data[fet][idx]):            
            # l = []
            # Iterate over data again and select an index and date of the data instance
            for idx2, date2 in data["date"].iteritems():
                # Check if the instances are from the same sampling station
                if (data["sampling station"][idx2] == data["sampling station"][idx]):
                    # Calculate the timespan between the two instances
                    timespan = date - date2
                    if min_range <= timespan <= max_range:
                        # l.append(idx2) 
                        data_ipl.loc[idx, fet] = data[fet][idx2]
                    else:
                        continue
                else:
                    continue      
        else:
            continue

In [124]:
data_ipl.isnull().sum()

date                       0
sampling station           0
DSP                        1
DSP_like                  60
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    422
salinity                  21
T                         59
SECCHI                   450
DIN                      352
PO4-P                    349
temp_dif                 346
Soca                       0
month                      0
lipophylic_toxins        320
dtype: int64

## Prepare original data for ML

In [ ]:
# dataframe with selected features
data.drop(columns=["date", "sampling station", "DSP_like", "temp_dif"], inplace=True)#
# data.isnull().sum()

# set date as index for slicing
# data_tree["org_idx"] = df_ipl_allF_L.index
# data_tree.set_index('date', inplace=True)

In [121]:
# Class distribution
data["lipophylic_toxins"].value_counts(dropna=False)

neg    996
NaN    320
poz    136
Name: lipophylic_toxins, dtype: int64

In [123]:
# EXPORT df with all features
# Create pickle
data.to_pickle(f"data/preprocessed/hab_org-data-{nb_fname}")
# Export csv
data.to_csv(f"data/preprocessed/hab_org-data-{nb_fname}.csv", sep=",", na_rep="?", index=False)

## Prepare interpolated data for ML

In [133]:
# All features
# dataframe with selected features
data_ipl = df_ipl.copy()
data_ipl.drop(columns=["date", "sampling station", "DSP_like", "temp_dif"], inplace=True)#
data_ipl.isnull().sum()

# set date as index for slicing
# df_ipl_allF_L["org_idx"] = df_ipl_allF_L.index
# df_ipl_allF.set_index('date', inplace=True)

DSP                        1
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    422
salinity                  21
T                         59
SECCHI                   450
DIN                      352
PO4-P                    349
Soca                       0
month                      0
lipophylic_toxins        320
dtype: int64

In [134]:
# Class distribution
data_ipl["lipophylic_toxins"].value_counts(dropna=False)

neg    996
NaN    320
poz    136
Name: lipophylic_toxins, dtype: int64

In [135]:
# EXPORT interpolated df with all features
# Create pickle
data_ipl.to_pickle(f"data/preprocessed/hab_interp_data-{nb_fname}")
# Export csv
data_ipl.to_csv(f"data/preprocessed/hab_interp_data-{nb_fname}.csv", sep=",", na_rep="?", index=False)